In [99]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from utilities.data_preprocessors import read_preprocess, series_to_1D_array

In [100]:
# 1 for religious and 0 for non religious
df = pd.read_csv('./data/hate-speech-data-cleaned.csv', index_col=0)
df = read_preprocess(df)

# before joining again get array in df with longest length first
max_len_1 = len(max(df['comment'], key=len))

In [101]:
all_words = pd.Series(series_to_1D_array(df['comment']))
all_unique_words_counts = all_words.value_counts()
all_unique_words = all_words.unique()

In [102]:
len(all_words)

894878

In [103]:
len(all_unique_words)

47921

In [104]:
all_unique_words_counts

nigger         16186
faggot         14812
bitch          12246
tranny         11850
like           11657
               ...  
pty                1
vocorp             1
sometimesin        1
shon               1
maupin             1
Length: 47921, dtype: int64

In [105]:
df['comment'] = df['comment'].apply(lambda comment: " ".join(comment))
df

,comment,label
0,woman complain cleaning house man always take ...,1
1,boy dat coldtyga dwn bad cuffin dat hoe st place,0
2,dawg ever fuck bitch start cry confused shit,0
3,look like tranny,0
4,shit hear might true might faker bitch told ya,0
...,...,...
65775,from the midnight sun where the hot spring blow,1
65776,do not say am not your type,1
65777,and therefor never send to know for whom the b...,1
65778,and cannot stand anoth day,1


**A note on the subsequent code below**

fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 1; word_index["cat"] = 2 it is word -> index dictionary so every word gets a unique integer value. 0 is reserved for padding. So lower integer means more frequent word (often the first few are stop words because they appear a lot).

texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.

In [106]:
train_sents, test_sents, train_labels, test_labels = train_test_split(df['comment'], df['label'], test_size=0.3, random_state=0)

max_len_2 = 50

num_words_1 = df.shape[0]
num_words_2 = len(all_words)
num_words_3 = len(all_unique_words)

tokenizer = Tokenizer(num_words=num_words_3)
tokenizer.fit_on_texts(train_sents)

train_seqs = tokenizer.texts_to_sequences(train_sents)
test_seqs = tokenizer.texts_to_sequences(test_sents)

moreover num_words of the Tokenizer can be an arbitrary number most likely based off of an educated guess like the number of sentences in the dataset itself

another is using the number of words itself in the dataset

another is even narrowing the nuber of words in the dataset by using only the number of uniquely occuring words in the dataset

<img src="./figures%20%26%20images/Nhwur.png">

In [107]:
train_seqs

[[81, 35, 16, 3, 61, 2846],
 [189, 612, 330, 151, 21, 1471, 4, 3037, 1954, 17949, 158, 3243],
 [1529, 19, 673, 85, 2, 13183],
 [6, 660, 599, 4745, 15, 269, 94, 147, 686, 14, 82, 340, 225],
 [2, 2847, 2765, 4294, 713, 1344, 185, 860],
 [60,
  2155,
  227,
  17950,
  2246,
  60,
  79,
  13184,
  2156,
  322,
  5331,
  613,
  2394,
  549,
  2612],
 [199, 1028, 3733, 3],
 [18, 2, 661, 17951, 20],
 [226, 481, 305, 372, 163, 35, 10719, 1106, 2942, 3, 363, 10720, 35],
 [614, 131, 20, 2330, 470, 4, 364, 4488, 6633, 294],
 [259, 12, 2, 129],
 [252, 53, 430, 36, 9, 2],
 [1548, 2613, 24, 179, 3, 48],
 [5, 13185, 649, 13186, 25, 1828, 12, 352, 9154, 17952],
 [118, 3351, 2943, 1, 17953, 168, 867, 1395, 13187],
 [44, 15, 130, 71, 165, 1170, 288, 89, 4, 803, 17954, 1345, 640, 968],
 [28, 545, 128, 14],
 [1118,
  2453,
  2034,
  193,
  5671,
  5672,
  1107,
  61,
  125,
  493,
  50,
  667,
  837,
  9155,
  77,
  1107,
  266,
  585,
  619,
  2848,
  918,
  762,
  53,
  950,
  187,
  10721,
  68,
  714,

In [108]:
len(train_seqs)

46046

In [109]:
len(test_seqs)

19734

Here we see that indeed 50 is not enough as our max length but for the subsequent code we will still use 50 and later 503 for our experimentation. For now 503 will be an extremely large value eespecially when applied to all sequences

In [110]:
print(max_len_1, max_len_2)

503 50


In [111]:
# post means place padding of 0's on the tail or ending of the sequence
# and truncating removes the values of a sequence that is greater than the max length given
train_seqs_padded = pad_sequences(train_seqs, maxlen=max_len_2, padding='post', truncating='post')
test_seqs_padded = pad_sequences(test_seqs, maxlen=max_len_2, padding='post', truncating='post')

In [112]:
print(train_seqs_padded[0])
print(test_seqs_padded[0])

array([[   81,    35,    16, ...,     0,     0,     0],
       [  189,   612,   330, ...,     0,     0,     0],
       [ 1529,    19,   673, ...,     0,     0,     0],
       ...,
       [ 7013,  2709,   139, ...,     0,     0,     0],
       [   92, 39261,   103, ...,     0,     0,     0],
       [  259,  2104,     1, ...,     0,     0,     0]])

In [113]:
len(train_seqs_padded)

46046

In [114]:
word_index_dict = tokenizer.word_index
word_index_dict

{'nigger': 1,
 'faggot': 2,
 'bitch': 3,
 'tranny': 4,
 'like': 5,
 'people': 6,
 'would': 7,
 'get': 8,
 'word': 9,
 'say': 10,
 'one': 11,
 'fuck': 12,
 'black': 13,
 'hoe': 14,
 'know': 15,
 'shit': 16,
 'think': 17,
 'fucking': 18,
 'call': 19,
 'time': 20,
 'guy': 21,
 'make': 22,
 'white': 23,
 'got': 24,
 'gay': 25,
 'want': 26,
 'even': 27,
 'go': 28,
 'u': 29,
 'called': 30,
 'someone': 31,
 'really': 32,
 'thing': 33,
 'said': 34,
 'nigga': 35,
 'use': 36,
 'right': 37,
 'as': 38,
 'pussy': 39,
 'look': 40,
 'saying': 41,
 'trannies': 42,
 'woman': 43,
 'see': 44,
 'racist': 45,
 'good': 46,
 'way': 47,
 'man': 48,
 'cannot': 49,
 'mean': 50,
 'calling': 51,
 'never': 52,
 'still': 53,
 'also': 54,
 'lol': 55,
 'person': 56,
 'hate': 57,
 'much': 58,
 'could': 59,
 'going': 60,
 'need': 61,
 'used': 62,
 'year': 63,
 'love': 64,
 'back': 65,
 'day': 66,
 'op': 67,
 'well': 68,
 'girl': 69,
 'something': 70,
 'bad': 71,
 'friend': 72,
 'let': 73,
 'take': 74,
 'actually': 75,
